In [1]:
!pip install -q efficientnet
!pip install -q tensorflow-addons
import re
!pip install -q git+https: // github.com // Kevin-McIsaac / cmorlet-tensorflow @ Performance --no-deps
import os
import numpy as np
import pandas as pd
from scipy.signal import get_window
from typing import Optional, Tuple
import warnings
import random
import math
import tensorflow as tf
import efficientnet.tfkeras as efn
from sklearn import metrics
from sklearn.model_selection import KFold, StratifiedKFold
from tensorflow.keras import backend as K
from tensorflow.keras import mixed_precision
import tensorflow_addons as tfa


     |████████████████████████████████| 50 kB 4.6 MB/s 
     |████████████████████████████████| 1.1 MB 5.3 MB/s 


In [2]:
# Function to get hardware strategy
def get_hardware_strategy():
	try:
		# TPU detection. No parameters necessary if TPU_NAME environment variable is
		# set: this is always the case on Kaggle.
		tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
		print('Running on TPU ', tpu.master())
	except ValueError:
		tpu = None

	if tpu:
		tf.config.experimental_connect_to_cluster(tpu)
		tf.tpu.experimental.initialize_tpu_system(tpu)
		strategy = tf.distribute.experimental.TPUStrategy(tpu)
		policy = mixed_precision.Policy('mixed_bfloat16')
		mixed_precision.set_global_policy(policy)
		tf.config.optimizer.set_jit(True)
	else:
		# Default distribution strategy in Tensorflow. Works on CPU and single GPU.
		strategy = tf.distribute.get_strategy()

	print("REPLICAS: ", strategy.num_replicas_in_sync)
	return tpu, strategy


tpu, strategy = get_hardware_strategy()

Running on TPU  grpc://10.99.48.74:8470
INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.99.48.74:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.99.48.74:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [3]:
# For tf.dataset
AUTO = tf.data.experimental.AUTOTUNE

# Data access (Train tf records)
GCS_PATH1 = "gs://kds-be973b4d24ed59842deeff7fc17c65f940acfe025d5bb2eb03261137"
GCS_PATH2 = "gs://kds-2a81609a438ac9f10d51ab54625a7f21f167acc05702fed3bfa405ab"
GCS_PATH3 = "gs://kds-9f5ea0ead6e9f5958187b731b5132e52b919b3173b744a0dc7c0c93c"
# Data access (Test tf records)
GCS_PATH4 = "gs://kds-5bd625816de42dc656f7ef6832ceedc0fd66cea6f4b639bf1489f4f0"
GCS_PATH5 = "gs://kds-42a3dac2586f0971b2a3b30162eed7cea42f2f9c34cbe66bd6c46b23"

EPOCHS = 18
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
IMAGE_SIZE = [512, 512]
# Seed
SEED = 1991
# Learning rate
LR = 0.0001
# Verbosity
VERBOSE = 2

# Training filenames directory
TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH1 + '/train*.tfrec') + tf.io.gfile.glob(
	GCS_PATH2 + '/train*.tfrec') + tf.io.gfile.glob(GCS_PATH3 + '/train*.tfrec')
# Testing filenames directory
TESTING_FILENAMES = tf.io.gfile.glob(GCS_PATH4 + '/test*.tfrec') + tf.io.gfile.glob(GCS_PATH5 + '/test*.tfrec')

In [4]:
# Function to create cqt kernel
def create_cqt_kernels(
		q: float,
		fs: float,
		fmin: float,
		n_bins: int = 84,
		bins_per_octave: int = 12,
		norm: float = 1,
		window: str = "hann",
		fmax: Optional[float] = None,
		topbin_check: bool = True
) -> Tuple[np.ndarray, int, np.ndarray, float]:
	fft_len = 2 ** _nextpow2(np.ceil(q * fs / fmin))

	if (fmax is not None) and (n_bins is None):
		n_bins = np.ceil(bins_per_octave * np.log2(fmax / fmin))
		freqs = fmin * 2.0 ** (np.r_[0:n_bins] / np.float(bins_per_octave))
	elif (fmax is None) and (n_bins is not None):
		freqs = fmin * 2.0 ** (np.r_[0:n_bins] / np.float(bins_per_octave))
	else:
		warnings.warn("If nmax is given, n_bins will be ignored", SyntaxWarning)
		n_bins = np.ceil(bins_per_octave * np.log2(fmax / fmin))
		freqs = fmin * 2.0 ** (np.r_[0:n_bins] / np.float(bins_per_octave))

	if np.max(freqs) > fs / 2 and topbin_check:
		raise ValueError(f"The top bin {np.max(freqs)} Hz has exceeded the Nyquist frequency, \
                           please reduce the `n_bins`")

	kernel = np.zeros((int(n_bins), int(fft_len)), dtype=np.complex64)

	length = np.ceil(q * fs / freqs)
	for k in range(0, int(n_bins)):
		freq = freqs[k]
		l = np.ceil(q * fs / freq)

		if l % 2 == 1:
			start = int(np.ceil(fft_len / 2.0 - l / 2.0)) - 1
		else:
			start = int(np.ceil(fft_len / 2.0 - l / 2.0))

		sig = get_window(window, int(l), fftbins=True) * np.exp(
			np.r_[-l // 2:l // 2] * 1j * 2 * np.pi * freq / fs) / l

		if norm:
			kernel[k, start:start + int(l)] = sig / np.linalg.norm(sig, norm)
		else:
			kernel[k, start:start + int(l)] = sig
	return kernel, fft_len, length, freqs


class BasicConv(tf.keras.Model):
	def __init__(self, out_planes, kernel_size, stride=1, padding="same", dilation=1, groups=1, relu=True,
	             bn=True, bias=False):
		super(BasicConv, self).__init__()

		self.out_channels = out_planes
		self.conv = tf.keras.layers.Conv2D(filters=out_planes, kernel_size=kernel_size, strides=stride, padding=padding,
		                                   dilation_rate=dilation, groups=groups, use_bias=bias)
		self.bn = tf.keras.layers.BatchNormalization(epsilon=1e-5, momentum=0.01, fused=True) if bn else None
		self.relu = tf.keras.layers.ReLU() if relu else None

	def call(self, x):
		x = self.conv(x)
		if self.bn is not None:
			x = self.bn(x)
		if self.relu is not None:
			x = self.relu(x)
		return x


class ChannelPool(tf.keras.Model):
	def __init__(self):
		super(ChannelPool, self).__init__()

	def __call__(self, x):
		return tf.reduce_max(x, axis=[3], keepdims=True)


class SpatialGate(tf.keras.Model):
	def __init__(self):
		super(SpatialGate, self).__init__()
		kernel_size = 7
		self.compress = ChannelPool()
		self.spatial = BasicConv(out_planes=1, kernel_size=kernel_size, stride=1, relu=False)

	def __call__(self, x):
		x_compress = self.compress(x)
		x_out = self.spatial(x_compress)
		scale = tf.math.sigmoid(x_out)
		return x * scale


class TripletAttention(tf.keras.Model):
	def __init__(self, no_spatial=False):
		super(TripletAttention, self).__init__()
		self.ChannelGateH = SpatialGate()
		self.ChannelGateW = SpatialGate()
		self.no_spatial = no_spatial
		if not no_spatial:
			self.SpatialGate = SpatialGate()

	def call(self, x):
		x_perm1 = tf.transpose(x, perm=(0, 2, 1, 3))
		x_out1 = self.ChannelGateH(x_perm1)
		x_out11 = tf.transpose(x_out1, (0, 2, 1, 3))
		x_perm2 = tf.transpose(x, perm=(0, 3, 2, 1))
		x_out2 = self.ChannelGateW(x_perm2)
		x_out21 = tf.transpose(x_out2, (0, 3, 2, 1))
		if not self.no_spatial:
			x_out = self.SpatialGate(x)
			x_out = (1 / 3) * (x_out + x_out11 + x_out21)
		else:
			x_out = (1 / 2) * (x_out11 + x_out21)
		return x_out


class GeMPoolingLayer(tf.keras.layers.Layer):
	def __init__(self, p=1., train_p=False):
		super().__init__()
		if train_p:
			self.p = tf.Variable(p, dtype=tf.float32)
		else:
			self.p = p
		self.eps = 1e-6

	def call(self, inputs: tf.Tensor, **kwargs):
		inputs = tf.clip_by_value(inputs, clip_value_min=1e-6, clip_value_max=tf.reduce_max(inputs))
		inputs = tf.pow(inputs, self.p)
		inputs = tf.reduce_mean(inputs, axis=[1, 2], keepdims=False)
		inputs = tf.pow(inputs, 1. / self.p)
		return inputs


model_aug = TripletAttention()


def _nextpow2(a: float) -> int:
	return int(np.ceil(np.log2(a)))


# Function to prepare cqt kernel
def prepare_cqt_kernel(
		sr=22050,
		hop_length=512,
		fmin=32.70,
		fmax=None,
		n_bins=84,
		bins_per_octave=12,
		norm=1,
		filter_scale=1,
		window="hann"
):
	q = float(filter_scale) / (2 ** (1 / bins_per_octave) - 1)
	print(q)
	return create_cqt_kernels(q, sr, fmin, n_bins, bins_per_octave, norm, window, fmax)


# Function to create cqt image
def create_cqt_image(wave, hop_length=16):
	CQTs = []
	for i in range(3):
		x = wave[i]
		x = tf.expand_dims(tf.expand_dims(x, 0), 2)
		x = tf.pad(x, PADDING, "REFLECT")

		CQT_real = tf.nn.conv1d(x, CQT_KERNELS_REAL, stride=hop_length, padding="VALID")
		CQT_imag = -tf.nn.conv1d(x, CQT_KERNELS_IMAG, stride=hop_length, padding="VALID")
		CQT_real *= tf.math.sqrt(LENGTHS)
		CQT_imag *= tf.math.sqrt(LENGTHS)

		CQT = tf.math.sqrt(tf.pow(CQT_real, 2) + tf.pow(CQT_imag, 2))
		CQTs.append(CQT[0])
	return tf.stack(CQTs, axis=2)


HOP_LENGTH = 6
cqt_kernels, KERNEL_WIDTH, lengths, _ = prepare_cqt_kernel(
	sr=2048,
	hop_length=HOP_LENGTH,
	fmin=20,
	fmax=1024,
	bins_per_octave=9)
LENGTHS = tf.constant(lengths, dtype=tf.float32)
CQT_KERNELS_REAL = tf.constant(np.swapaxes(cqt_kernels.real[:, np.newaxis, :], 0, 2))
CQT_KERNELS_IMAG = tf.constant(np.swapaxes(cqt_kernels.imag[:, np.newaxis, :], 0, 2))
PADDING = tf.constant([[0, 0],
                       [KERNEL_WIDTH // 2, KERNEL_WIDTH // 2],
                       [0, 0]])

12.490672763062207


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SyntaxWarning: If nmax is given, n_bins will be ignored


In [5]:
def seed_everything(seed):
	random.seed(seed)
	np.random.seed(seed)
	os.environ['PYTHONHASHSEED'] = str(seed)
	tf.random.set_seed(seed)


# Function to prepare image
# Function to prepare image
def prepare_image(wave):
	# Decode raw
	wave = tf.reshape(tf.io.decode_raw(wave, tf.float64), (3, 4096))
	normalized_waves = []
	# Normalize
	for i in range(3):
		normalized_wave = wave[i] / tf.math.reduce_max(wave[i])
		normalized_waves.append(normalized_wave)
	# Stack and cast
	wave = tf.stack(normalized_waves)
	wave = tf.cast(wave, tf.float32)
	# Create image
	image = create_cqt_image(wave, HOP_LENGTH)
	image = tf.convert_to_tensor(model_aug(tf.expand_dims(image, axis=1)))
	# Resize image
	image = tf.image.resize(image, [*IMAGE_SIZE])
	image = tf.squeeze(tfa.image.random_cutout(image, mask_size=10, constant_values=255))
	image = image[0, :, :, :]

	# Reshape
	image = tf.reshape(image, [*IMAGE_SIZE, 3])
	return tf.cast(image, tf.bfloat16)


# This function parse our images and also get the target variable
def read_labeled_tfrecord(example):
	LABELED_TFREC_FORMAT = {
		'wave': tf.io.FixedLenFeature([], tf.string),
		'wave_id': tf.io.FixedLenFeature([], tf.string),
		'target': tf.io.FixedLenFeature([], tf.int64)
	}
	example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
	image = prepare_image(example['wave'])
	image_id = example['wave_id']
	target = tf.cast(example['target'], tf.float32)
	return image, image_id, target


# This function parse our images and also get the target variable
def read_unlabeled_tfrecord(example):
	LABELED_TFREC_FORMAT = {
		'wave': tf.io.FixedLenFeature([], tf.string),
		'wave_id': tf.io.FixedLenFeature([], tf.string)
	}
	example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
	image = prepare_image(example['wave'])
	image_id = example['wave_id']
	return image, image_id


# This function loads TF Records and parse them into tensors
def load_dataset(filenames, ordered=False, labeled=True):
	ignore_order = tf.data.Options()
	if not ordered:
		ignore_order.experimental_deterministic = False

	dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
	dataset = dataset.with_options(ignore_order)
	dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls=AUTO)
	return dataset


# This function is to get our training dataset
def get_training_dataset(filenames, ordered=False, labeled=True):
	dataset = load_dataset(filenames, ordered=ordered, labeled=labeled)
	dataset = dataset.repeat()
	dataset = dataset.shuffle(2048)
	dataset = dataset.batch(BATCH_SIZE)
	dataset = dataset.prefetch(AUTO)
	return dataset


# This function is to get our validation and test dataset
def get_val_test_dataset(filenames, ordered=True, labeled=True):
	dataset = load_dataset(filenames, ordered=ordered, labeled=labeled)
	dataset = dataset.batch(BATCH_SIZE)
	dataset = dataset.prefetch(AUTO)
	return dataset


# Function to count how many photos we have in
def count_data_items(filenames):
	# The number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
	n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
	return np.sum(n)


NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
NUM_TESTING_IMAGES = count_data_items(TESTING_FILENAMES)
print(f'Dataset: {NUM_TRAINING_IMAGES} training images')
print(f'Dataset: {NUM_TESTING_IMAGES} testing images')

Dataset: 560000 training images
Dataset: 226000 testing images


In [ ]:
# Learning rate callback function
def get_lr_callback():
	lr_start = 0.0001
	lr_max = 0.000015 * BATCH_SIZE
	lr_min = 0.0000001
	lr_ramp_ep = 3
	lr_sus_ep = 0
	lr_decay = 0.7

	def lrfn(epoch):
		if epoch < lr_ramp_ep:
			lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
		elif epoch < lr_ramp_ep + lr_sus_ep:
			lr = lr_max
		else:
			lr = (lr_max - lr_min) * lr_decay ** (epoch - lr_ramp_ep - lr_sus_ep) + lr_min
		return lr

	lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=VERBOSE)
	return lr_callback


# Function to create our EfficientNetB7 model
def get_model():
	with strategy.scope():
		inp = tf.keras.layers.Input(shape=(*IMAGE_SIZE, 3))
		x = efn.EfficientNetB7(include_top=False, weights='imagenet')(inp)
		x = GeMPoolingLayer(p=None,)(x)
		output = tf.keras.layers.Dense(1, activation='sigmoid')(x)
		model = tf.keras.models.Model(inputs=[inp], outputs=[output])
		opt = tf.keras.optimizers.Adam(learning_rate=LR)
		opt = tfa.optimizers.SWA(opt)
		model.compile(
			optimizer=opt,
			loss=[tf.keras.losses.BinaryCrossentropy()],
			metrics=[tf.keras.metrics.AUC()]
		)
		return model


options = tf.train.CheckpointOptions(experimental_io_device="/job:localhost")
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
	filepath="content/Temp",
	save_weights_only=True,
	monitor='auc',
	mode='max',
	save_best_only=True, options=options)


# Function to train a model with 100% of the data
def train_and_evaluate(SEED=42):
	print('\n')
	print('-' * 50)
	print(f'Training EFFB7 with 100% of the data with seed {SEED} for {EPOCHS} epochs')
	if tpu:
		tf.tpu.experimental.initialize_tpu_system(tpu)
	train_dataset = get_training_dataset(TRAINING_FILENAMES, ordered=False, labeled=True)
	train_dataset = train_dataset.map(lambda image, image_id, target: (image, target))
	STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // (BATCH_SIZE * 4)
	K.clear_session()
	# Seed everything
	seed_everything(SEED)
	model = get_model()
	history = model.fit(train_dataset,
	                    steps_per_epoch=STEPS_PER_EPOCH,
	                    epochs=EPOCHS,
	                    callbacks=[get_lr_callback(), model_checkpoint_callback],
	                    verbose=1)

	print('\n')
	print('-' * 50)
	print('Test inference...')
	model = model.load_weights("content/Temp", options=options)
	# Predict the test set
	dataset = get_val_test_dataset(TESTING_FILENAMES, ordered=True, labeled=False)
	image = dataset.map(lambda image, image_id: image)
	test_predictions = model.predict(image).astype(np.float32).reshape(-1)
	# Get the test set image_id
	image_id = dataset.map(lambda image, image_id: image_id).unbatch()
	image_id = next(iter(image_id.batch(NUM_TESTING_IMAGES))).numpy().astype('U')
	# Create dataframe output
	test_df = pd.DataFrame({'id': image_id, 'target': test_predictions})
	# Save test dataframe to disk
	test_df.to_csv(f'TEST_EfficientNetB7_{IMAGE_SIZE[0]}_{SEED}.csv', index=False)


train_and_evaluate(SEED=2020)
g



--------------------------------------------------
Training EFFB7 with 100% of the data with seed 2020 for 18 epochs
INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.99.48.74:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.99.48.74:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


Epoch 1/18

Epoch 00001: LearningRateScheduler setting learning rate to 0.0001.
1093/1093 [==============================] - 1695s 1s/step - loss: 0.4984 - auc: 0.8098
Epoch 2/18

Epoch 00002: LearningRateScheduler setting learning rate to 0.0007066666666666667.
1093/1093 [==============================] - 1152s 1s/step - loss: 0.4601 - auc: 0.8354
Epoch 3/18

Epoch 00003: LearningRateScheduler setting learning rate to 0.0013133333333333335.
1093/1093 [==============================] - 1153s 1s/step - loss: 0.4474 - auc: 0.8448
Epoch 4/18

Epoch 00004: LearningRateScheduler setting learning rate to 0.00192.
1093/1093 [==============================] - 1152s 1s/step - loss: 0.4414 - auc: 0.8496
Epoch 5/18

Epoch 00005: LearningRateScheduler setting learning rate to 0.00134403.
1093/1093 [==============================] - 1152s 1s/step - loss: 0.4303 - auc: 0.8573
Epoch 6/18

Epoch 00006: LearningRateScheduler setting learning rate to 0.0009408509999999998.
1093/1093 [===================

In [ ]:
print(f'TEST_EfficientNetB7_{IMAGE_SIZE[0]}_{SEED}.csv')

In [ ]:
!cp -r "TEST_EfficientNetB7_512_1991.csv" "/content/drive/MyDrive/models"

In [ ]:
from google.colab import drive

drive.mount('/content/drive')
model = model.load_weights("/content/content/Temp", options=options)
# Predict the test set
dataset = get_val_test_dataset(TESTING_FILENAMES, ordered=True, labeled=False)
image = dataset.map(lambda image, image_id: image)
test_predictions = model.predict(image).astype(np.float32).reshape(-1)
# Get the test set image_id
image_id = dataset.map(lambda image, image_id: image_id).unbatch()
image_id = next(iter(image_id.batch(NUM_TESTING_IMAGES))).numpy().astype('U')
# Create dataframe output
test_df = pd.DataFrame({'id': image_id, 'target': test_predictions})
# Save test dataframe to disk
test_df.to_csv(f'TEST_EfficientNetB7_{IMAGE_SIZE[0]}_{SEED}.csv', index=False)